In [2]:
import numpy as np, sys
np.random.seed(1)

from keras.datasets import mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()

images, labels = (x_train[0:1000].reshape(1000,28*28) / 255,
                  y_train[0:1000])


one_hot_labels = np.zeros((len(labels),10))
for i,l in enumerate(labels):
    one_hot_labels[i][l] = 1
labels = one_hot_labels

test_images = x_test.reshape(len(x_test),28*28) / 255
test_labels = np.zeros((len(y_test),10))
for i,l in enumerate(y_test):
    test_labels[i][l] = 1

def tanh(x):
    return np.tanh(x)

def tanh2deriv(output):
    return 1 - (output ** 2)

def softmax(x):
    temp = np.exp(x)
    return temp / np.sum(temp, axis=1, keepdims=True)

alpha, iterations = (2, 300)
pixels_per_image, num_labels = (784, 10)
batch_size = 128

input_rows = 28
input_cols = 28

kernel_rows = 3
kernel_cols = 3
num_kernels = 16

hidden_size = ((input_rows - kernel_rows) * 
               (input_cols - kernel_cols)) * num_kernels

# weights_0_1 = 0.02*np.random.random((pixels_per_image,hidden_size))-0.01
kernels = 0.02*np.random.random((kernel_rows*kernel_cols,
                                 num_kernels))-0.01

weights_1_2 = 0.2*np.random.random((hidden_size,
                                    num_labels)) - 0.1



def get_image_section(layer,row_from, row_to, col_from, col_to):
    section = layer[:,row_from:row_to,col_from:col_to]
    return section.reshape(-1,1,row_to-row_from, col_to-col_from)

for j in range(iterations):
    correct_cnt = 0
    for i in range(int(len(images) / batch_size)):
        batch_start, batch_end=((i * batch_size),((i+1)*batch_size))
        layer_0 = images[batch_start:batch_end]
        layer_0 = layer_0.reshape(layer_0.shape[0],28,28)
        layer_0.shape

        sects = list()
        for row_start in range(layer_0.shape[1]-kernel_rows):
            for col_start in range(layer_0.shape[2] - kernel_cols):
                sect = get_image_section(layer_0,
                                         row_start,
                                         row_start+kernel_rows,
                                         col_start,
                                         col_start+kernel_cols)
                sects.append(sect)

        expanded_input = np.concatenate(sects,axis=1)
        es = expanded_input.shape
        flattened_input = expanded_input.reshape(es[0]*es[1],-1)

        kernel_output = flattened_input.dot(kernels)
        layer_1 = tanh(kernel_output.reshape(es[0],-1))
        dropout_mask = np.random.randint(2,size=layer_1.shape)
        layer_1 *= dropout_mask * 2
        layer_2 = softmax(np.dot(layer_1,weights_1_2))

        for k in range(batch_size):
            labelset = labels[batch_start+k:batch_start+k+1]
            _inc = int(np.argmax(layer_2[k:k+1]) == 
                               np.argmax(labelset))
            correct_cnt += _inc

        layer_2_delta = (labels[batch_start:batch_end]-layer_2)\
                        / (batch_size * layer_2.shape[0])
        layer_1_delta = layer_2_delta.dot(weights_1_2.T) * \
                        tanh2deriv(layer_1)
        layer_1_delta *= dropout_mask
        weights_1_2 += alpha * layer_1.T.dot(layer_2_delta)
        l1d_reshape = layer_1_delta.reshape(kernel_output.shape)
        k_update = flattened_input.T.dot(l1d_reshape)
        kernels -= alpha * k_update
    
    test_correct_cnt = 0

    for i in range(len(test_images)):

        layer_0 = test_images[i:i+1]
#         layer_1 = tanh(np.dot(layer_0,weights_0_1))
        layer_0 = layer_0.reshape(layer_0.shape[0],28,28)
        layer_0.shape

        sects = list()
        for row_start in range(layer_0.shape[1]-kernel_rows):
            for col_start in range(layer_0.shape[2] - kernel_cols):
                sect = get_image_section(layer_0,
                                         row_start,
                                         row_start+kernel_rows,
                                         col_start,
                                         col_start+kernel_cols)
                sects.append(sect)

        expanded_input = np.concatenate(sects,axis=1)
        es = expanded_input.shape
        flattened_input = expanded_input.reshape(es[0]*es[1],-1)

        kernel_output = flattened_input.dot(kernels)
        layer_1 = tanh(kernel_output.reshape(es[0],-1))
        layer_2 = np.dot(layer_1,weights_1_2)

        test_correct_cnt += int(np.argmax(layer_2) == 
                                np.argmax(test_labels[i:i+1]))
        sys.stdout.write("\n"+ \
         "I:" + str(j) + \
         " Test-Acc:"+str(test_correct_cnt/float(len(test_images)))+\
         " Train-Acc:" + str(correct_cnt/float(len(images))))



I:0 Test-Acc:0.0 Train-Acc:0.055
I:0 Test-Acc:0.0 Train-Acc:0.055
I:0 Test-Acc:0.0 Train-Acc:0.055
I:0 Test-Acc:0.0 Train-Acc:0.055
I:0 Test-Acc:0.0 Train-Acc:0.055
I:0 Test-Acc:0.0 Train-Acc:0.055
I:0 Test-Acc:0.0 Train-Acc:0.055
I:0 Test-Acc:0.0 Train-Acc:0.055
I:0 Test-Acc:0.0 Train-Acc:0.055
I:0 Test-Acc:0.0 Train-Acc:0.055
I:0 Test-Acc:0.0 Train-Acc:0.055
I:0 Test-Acc:0.0 Train-Acc:0.055
I:0 Test-Acc:0.0 Train-Acc:0.055
I:0 Test-Acc:0.0 Train-Acc:0.055
I:0 Test-Acc:0.0 Train-Acc:0.055
I:0 Test-Acc:0.0 Train-Acc:0.055
I:0 Test-Acc:0.0 Train-Acc:0.055
I:0 Test-Acc:0.0 Train-Acc:0.055
I:0 Test-Acc:0.0 Train-Acc:0.055
I:0 Test-Acc:0.0 Train-Acc:0.055
I:0 Test-Acc:0.0 Train-Acc:0.055
I:0 Test-Acc:0.0 Train-Acc:0.055
I:0 Test-Acc:0.0 Train-Acc:0.055
I:0 Test-Acc:0.0 Train-Acc:0.055
I:0 Test-Acc:0.0001 Train-Acc:0.055
I:0 Test-Acc:0.0001 Train-Acc:0.055
I:0 Test-Acc:0.0001 Train-Acc:0.055
I:0 Test-Acc:0.0001 Train-Acc:0.055
I:0 Test-Acc:0.0001 Train-Acc:0.055
I:0 Test-Acc:0.0001 Train-A

KeyboardInterrupt: 

In [3]:
def pretty_print_review_and_label(i):
   print(labels[i] + "\t:\t" + reviews[i][:80] + "...")

g = open('data/reviews.txt','r') # What we know!
reviews = list(map(lambda x:x[:-1],g.readlines()))
g.close()

g = open('data/labels.txt','r') # What we WANT to know!
labels = list(map(lambda x:x[:-1].upper(),g.readlines()))
g.close()

In [5]:
def pretty_print_review_and_label(i):
   print(labels[i] + "\t:\t" + reviews[i][:80] + "...")

g = open('data/reviews.txt','r') # What we know!
reviews = list(map(lambda x:x[:-1],g.readlines()))
g.close()

g = open('data/labels.txt','r') # What we WANT to know!
labels = list(map(lambda x:x[:-1].upper(),g.readlines()))
g.close()

In [9]:
import numpy as np

onehots = {}
onehots['cat'] = np.array([1,0,0,0])
onehots['the'] = np.array([0,1,0,0])
onehots['dog'] = np.array([0,0,1,0])
onehots['sat'] = np.array([0,0,0,1])

sentence = ['the','cat','sat']
x = onehots[sentence[0]] + \
    onehots[sentence[1]] + \
    onehots[sentence[2]]

print("Sent Encoding:" + str(x))

Sent Encoding:[1 1 0 1]


In [14]:
import sys

f = open('data/reviews.txt')
raw_reviews = f.readlines()
f.close()

f = open('data/labels.txt')
raw_labels = f.readlines()
f.close()

tokens = list(map(lambda x:set(x.split(" ")),raw_reviews))

vocab = set()
for sent in tokens:
    for word in sent:
        if(len(word)>0):
            vocab.add(word)
vocab = list(vocab)

In [15]:
word2index = {}
for i,word in enumerate(vocab):
    word2index[word]=i


In [27]:
input_dataset = list()
for sent in tokens:
    sent_indices = list()
    for word in sent:
        try:
            sent_indices.append(word2index[word])
        except:
            ""
    input_dataset.append(list(set(sent_indices)))

In [16]:
target_dataset = list()
for label in raw_labels:
    if label == 'positive\n':
        target_dataset.append(1)
    else:
        target_dataset.append(0)

In [34]:
import numpy as np 
np.random.seed(1) 

def sigmoid(x): 
    return 1/(1 + np.exp(-x)) 

alpha, iterations = (0.01, 2) 
hidden_size = 100 
weights_0_1 = 0.2*np.random.random((len(vocab),hidden_size)) - 0.1 
weights_1_2 = 0.2*np.random.random((hidden_size,1)) - 0.1 
correct,total = (0,0)

for iter in range(iterations): 
    # train on first 24,000 
    for i in range(len(input_dataset)-1000): 
        x,y = (input_dataset[i],target_dataset[i]) 
        layer_1 = sigmoid(np.sum(weights_0_1[x],axis=0)) #embed + sigmoid 
        layer_2 = sigmoid(np.dot(layer_1,weights_1_2)) # linear + softmax 
        layer_2_delta = layer_2 - y # compare pred with truth 
        layer_1_delta = layer_2_delta.dot(weights_1_2.T) #backprop 
        weights_0_1[x] -= layer_1_delta * alpha 
        weights_1_2 -= np.outer(layer_1,layer_2_delta) * alpha 
        if(np.abs(layer_2_delta) < 0.5): 
            correct += 1 
        total += 1 
            
correct,total = (0,0) 
for i in range(len(input_dataset)-1000,len(input_dataset)): 
    x = input_dataset[i] 
    y = target_dataset[i] 
    layer_1 = sigmoid(np.sum(weights_0_1[x],axis=0)) 
    layer_2 = sigmoid(np.dot(layer_1,weights_1_2)) 
    if(np.abs(layer_2 - y) < 0.5): 
        correct += 1 
        total += 1 
print("Test Accuracy:" + str(correct / float(total))) 

Test Accuracy:1.0


In [37]:
from collections import Counter
import math 

def similar(target='beautiful'):
    target_index = word2index[target]
    scores = Counter()
    for word,index in word2index.items():
        raw_difference = weights_0_1[index] - (weights_0_1[target_index])
        squared_difference = raw_difference * raw_difference
        scores[word] = -math.sqrt(sum(squared_difference))

    return scores.most_common(10)

In [38]:
print(similar('beautiful'))

[('beautiful', -0.0), ('touching', -0.697142777110716), ('cry', -0.7045784095955291), ('expecting', -0.706839931665644), ('believable', -0.709402340848879), ('ride', -0.715164626219281), ('brilliant', -0.7219674020152913), ('success', -0.7228211782431506), ('jack', -0.7234014219383437), ('appreciated', -0.7304435165380977)]


In [39]:
print(similar('terrible'))

[('terrible', -0.0), ('poor', -0.7417981110793378), ('boring', -0.7748335705425528), ('disappointment', -0.7863365524440662), ('worse', -0.7880657350204098), ('disappointing', -0.7881188962082628), ('lacks', -0.7919310628213366), ('laughable', -0.8149114464645917), ('mess', -0.8376177675524124), ('annoying', -0.844926132997756)]
